**Importing libraries**

In [139]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import nltk 
import random 
from nltk.tokenize import word_tokenize
nltk.download('punkt') 
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [140]:
import urllib.request as urllib2
from nltk import FreqDist
import pandas as pd
url = 'https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'
response = urllib2.urlopen(url)

df = pd.read_csv(response,delimiter='\t',encoding='utf-8')
df.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [141]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,56916,2863,in almost all of his previous works,2
1,46618,2268,do n't notice the 129-minute running time,3
2,133671,7211,"Think of it as Gidget , only with muscles and ...",4
3,24828,1133,is funny in the way that makes you ache with s...,2
4,35746,1686,need to see it,3


In [142]:
X_train, X_test, Y_train, Y_test = train_test_split(df ['Phrase'], df ['Sentiment'], test_size=0.3, random_state=2003)
documents=[]
X_train = np.array(X_train.values.tolist())
Y_train = np.array(Y_train.values.tolist())

for i in range(len(X_train)):
  documents.append([list(word_tokenize(X_train[i])), Y_train[i]]) 
  
X_test = np.array(X_test.values.tolist())
Y_test = np.array(Y_test.values.tolist())
for i in range(len(X_test)):
  documents.append([list(word_tokenize(X_test[i])), Y_test[i]]) 

documents[0]

[['it', "'s", 'like', 'dying', 'and', 'going', 'to', 'celluloid', 'heaven'], 3]

**Data pre-processing steps**

In [0]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()"
remove_stopwords = True
useStemming = False
useLemma = True
removePuncs = True


In [144]:
for l in range(len(documents)):                  
  label = documents[l][1]                       
  tmpReview = []                               
  for w in documents[l][0]:                       
    newWord = w                                   
    if remove_stopwords and (w in stopwords_en):   
      continue                                     
    if removePuncs and (w in punctuations):        
      continue                                    
    if useStemming:
      newWord = lancaster.stem(newWord) 
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord) 
    tmpReview.append(newWord)                      
  documents[l] = (tmpReview, label)              
  documents[l] = (' '.join(tmpReview), label) 
print(documents[0])

("'s like dying going celluloid heaven", 3)


In [145]:
df = pd.DataFrame(documents, columns=['text', 'sentiment']) 
df.head()

,text,sentiment
0,'s like dying going celluloid heaven,3
1,charitable,3
2,since `` Dumbo '',2
3,critical commercial disaster,0
4,The movie fails portray literarily talented no...,0


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],  df['sentiment'], test_size=0.3, random_state=2003)

**Vectorization**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical

vectorizer = TfidfVectorizer(max_features = 2500) 
X = vectorizer.fit_transform(df["text"]) 
Y = df['sentiment'] 
 
X_train = vectorizer.transform(X_train).toarray()
Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
Y_test = Y_test


In [148]:
Y_test

13510     1
61932     2
82549     2
137718    4
121990    4
         ..
94224     1
135456    1
154729    3
23031     2
57870     2
Name: sentiment, Length: 46818, dtype: int64

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K

In [0]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [151]:
X_train.shape

(109242, 2500)

In [0]:
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)
num_classes = 5

In [153]:
Y_test

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [0]:
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential

**Proposed model**

In [0]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1,
                 activation='relu',
                 input_shape=(2500,1)))
model.add(Conv1D(64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
#model.add(Dropout(rate = 0.50))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy',f1_m,precision_m, recall_m])

In [0]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

**Testing the model**

In [158]:
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=50)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Models loss:', score[0])
print('Models accuracy:', score[1])
print('Models F1 measure:',score[2])
print('Models precision:',score[3])
print('Models recall:',score[4])

Epoch 1/50
109242/109242 [==============================] - 22s 205us/step - loss: 1.0718 - acc: 0.5799 - f1_m: 0.5155 - precision_m: 0.6389 - recall_m: 0.4372
Epoch 2/50
109242/109242 [==============================] - 22s 201us/step - loss: 0.9676 - acc: 0.6169 - f1_m: 0.5828 - precision_m: 0.6765 - recall_m: 0.5132
Epoch 3/50
109242/109242 [==============================] - 22s 201us/step - loss: 0.9412 - acc: 0.6272 - f1_m: 0.5967 - precision_m: 0.6812 - recall_m: 0.5320
Epoch 4/50
109242/109242 [==============================] - 22s 201us/step - loss: 0.9243 - acc: 0.6318 - f1_m: 0.6032 - precision_m: 0.6831 - recall_m: 0.5412
Epoch 5/50
109242/109242 [==============================] - 22s 200us/step - loss: 0.9111 - acc: 0.6378 - f1_m: 0.6117 - precision_m: 0.6892 - recall_m: 0.5509
Epoch 6/50
109242/109242 [==============================] - 22s 200us/step - loss: 0.9012 - acc: 0.6399 - f1_m: 0.6152 - precision_m: 0.6897 - recall_m: 0.5562
Epoch 7/50
109242/109242 [==============

**Storing the model**

In [159]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
model.save('/content/drive/My Drive/1111419_1dconv_reg.h5')